![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime, timedelta
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

from pandas import Series, DataFrame
%matplotlib inline
def get_rest_count(value):
    return rest_count[value]


%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train= pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')




In [ ]:
# создаю словарь со странами и столицами
city_country = {'London': 'UK', 'Paris': 'France', 'Madrid': 'Spain', 'Barcelona': 'Spain',
                'Berlin': 'Germany', 'Milan': 'Italy', 'Rome': 'Italy', 'Prague': 'Czech',
                'Lisbon': 'Portugalia', 'Vienna': 'Austria', 'Amsterdam': 'Nederlands', 'Brussels': 'Belgium',
                'Hamburg': 'Germany', 'Munich': 'Germany', 'Lyon': 'France', 'Stockholm': 'Sweden',
                'Budapest': 'Hungary', 'Warsaw': 'Poland', 'Dublin': 'Ireland', 'Copenhagen': 'Denmark',
                'Athens': 'Greece', 'Edinburgh': 'Schotland', 'Zurich': 'Switzerland', 'Oporto': 'Portugalia',
                'Geneva': 'Switzerland', 'Krakow': 'Poland', 'Oslo': 'Norway', 'Helsinki': 'Finland',
                'Bratislava': 'Slovakia', 'Luxembourg': 'Luxembourg', 'Ljubljana': 'Slovenija'}

# создаю словарь, является ли город столицей
city_capital = {'London': 1, 'Paris': 1, 'Madrid': 1, 'Barcelona': 0, 'Berlin': 1,
                'Milan': 0, 'Rome': 1, 'Prague': 1, 'Lisbon': 1, 'Vienna': 1,
                'Amsterdam': 1, 'Brussels': 1, 'Hamburg': 0, 'Munich': 0, 'Lyon': 0,
                'Stockholm': 1, 'Budapest': 1, 'Warsaw': 1, 'Dublin': 1, 'Copenhagen': 1,
                'Athens': 1, 'Edinburgh': 1, 'Zurich': 1, 'Oporto': 0, 'Geneva': 1,
                'Krakow': 1, 'Oslo': 1, 'Helsinki': 1, 'Bratislava': 1, 'Luxembourg': 1, 'Ljubljana': 1}

# создаю словарь с информацией о населении города
city_population = {'London': 8173900, 'Paris': 2240621, 'Madrid': 3155360, 'Barcelona': 1593075,
                   'Berlin': 3326002, 'Milan': 1331586, 'Rome': 2870493, 'Prague': 1272690,
                   'Lisbon': 547733, 'Vienna': 1765649, 'Amsterdam': 825080, 'Brussels': 144784,
                   'Hamburg': 1718187, 'Munich': 1364920, 'Lyon': 496343, 'Stockholm': 1981263,
                   'Budapest': 1744665, 'Warsaw': 1720398, 'Dublin': 506211, 'Copenhagen': 1246611,
                   'Athens': 3168846, 'Edinburgh': 476100, 'Zurich': 402275, 'Oporto': 221800,
                   'Geneva': 196150, 'Krakow': 756183, 'Oslo': 673469, 'Helsinki': 574579,
                   'Bratislava': 413192, 'Luxembourg': 576249, 'Ljubljana': 277554}

# создаю словарь с указанием количества ресторанов для каждого города
res_count = {'Paris': 17593, 'Stockholm': 3131, 'London': 22366, 'Berlin': 8110,
             'Munich': 3367, 'Oporto': 2060, 'Milan': 7940, 'Bratislava': 1331,
             'Vienna': 4387, 'Rome': 12086, 'Barcelona': 10086, 'Madrid': 11562,
             'Dublin': 2706, 'Brussels': 3703, 'Zurich': 1901, 'Warsaw': 3210,
             'Budapest': 3445, 'Copenhagen': 2637, 'Amsterdam': 4189, 'Lyon': 2833,
             'Hamburg': 3501, 'Lisbon': 4985, 'Prague': 5850, 'Oslo': 1441,
             'Helsinki': 1661, 'Edinburgh': 2248, 'Geneva': 1753, 'Ljubljana': 647,
             'Athens': 2814, 'Luxembourg': 759, 'Krakow': 1832}



In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [ ]:
data['Number_of_Reviews_isNAN']

In [ ]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(0, inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [ ]:
data.head(5)

In [ ]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [ ]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [ ]:
# Ваша обработка 'Price Range'

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [ ]:
# тут ваш код на обработку других признаков
# .....

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_test['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train= pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews'].fillna(0, inplace=True)
    # тут ваш код по обработке NAN
    # ....
    
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    # тут ваш код не Encoding фитчей
    # ....
    
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # ....
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 
df=df_test
data = df.append(df_train, sort=False).reset_index(drop=True) # объединяем

# в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями
df=df_test
df.columns = map(lambda name: name.lower(), df.columns)
df.columns = df.columns.str.replace(' ', '_')

# Относительный ранг, чем больше к единице, тем выше ранг ресторана
rest_count = df.city.value_counts()


def get_rest_count(value):
    return rest_count[value]


df['relative_ranking'] = 1 - \
    (df['ranking'] / df['city'].map(df.groupby(['city'])['ranking'].max()))


# Добавляю столбец со страной
df['country'] = df['city'].map(city_country)

# Создаю признак, отражающий количество ресторанов в городе, в котором расположен данный ресторан
df['restaurants_count'] = df['city'].map(res_count)


# Создаю признак с населением города
df['population'] = df['city'].map(city_population)

# признак пропорция кол-ва ресторанов и населения
df['per_capital'] = df['population']/df['restaurants_count']

# Состоит ли ресторан в сети
restaurant_chain = set()
for chain in df['restaurant_id']:
    restaurant_chain.update(chain)
    

In [ ]:
df_train.columns = map(lambda name: name.lower(), df_train.columns)
df_train.columns = df_train.columns.str.replace(' ', '_')

# Относительный ранг, чем больше к единице, тем выше ранг ресторана
rest_count = df_train.city.value_counts()


def get_rest_count(value):
    return rest_count[value]


df_train['relative_ranking'] = 1 - \
    (df_train['ranking'] / df_train['city'].map(df_train.groupby(['city'])['ranking'].max()))


# Добавляю столбец со страной
df_train['country'] = df_train['city'].map(city_country)

# Создаю признак, отражающий количество ресторанов в городе, в котором расположен данный ресторан
df_train['restaurants_count'] = df_train['city'].map(res_count)


# Создаю признак с населением города
df_train['population'] = df_train['city'].map(city_population)

# признак пропорция кол-ва ресторанов и населения
df_train['per_capital'] = df_train['population']/df_train['restaurants_count']

# Состоит ли ресторан в сети
restaurant_chain = set()
for chain in df_train['restaurant_id']:
    restaurant_chain.update(chain)

In [ ]:
def find_item(cell):
    if item in cell:
        return 1
    return 0


for item in restaurant_chain:
    df['restaurant_chain'] = df['restaurant_id'].apply(find_item)

# Сколько городов представлено в наборе данных?
len(df['city'].unique())

# CUISINES

# столбец с кухнями разделяем на отдельные виды
df['cuisine_style'] = df.cuisine_style.str.replace(
    ']', '').str.replace('[', '')
o = df[['id_ta', 'cuisine_style']]
o.cuisine_style.fillna("'Europian'", inplace=True)
s = o['cuisine_style'].str.split(',').apply(Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'cuisine_style'
del o['cuisine_style']
k = o.join(s)

# Создаем признак "Кол-во кухонь в каждом ресторане"
table_with_num = k.copy()
table_with_num['num_of_cuis'] = (table_with_num.groupby(
    ['id_ta'])['cuisine_style']).transform('nunique')
table_with_num.drop(['cuisine_style'], axis=1, inplace=True)
table_with_num = table_with_num.drop_duplicates()
df = df.merge(table_with_num, how='left')

# среднее количество кухонь на ресторан в городе
df['cuisine_count_mean'] = df['city'].map(
    df.groupby('city')['num_of_cuis'].mean())

# Добавляю признак о наличии веганской кухни в ресторане

df['veg'] = df['cuisine_style'].str.contains(r'Veg', na=True)
df['veg'] = df['veg'].astype(int)
CuisinesDict = {}
# Функция заполнения словаря CuisinesDict


def CuisinesDictFill(cuisines):
    global CuisinesDict
    # из входящей строки вида "['German', 'Central European', 'Vegetarian Friendly']"
    cuisines = str(cuisines).replace("['", "").replace(
        "']", "").replace("', '", ",").split(',')
    # получили список отдельных значений вида ['German', 'Central European', 'Vegetarian Friendly']
    if cuisines[0] != 'nan':  # если список кухонь не пустой
        for i in cuisines:  # перебор полученного списка
            if CuisinesDict.get(i) == None:  # нет такой кухни в словаре CuisinesDict
                # добавить кухню (ключ) в CuisinesDict со значением 1
                CuisinesDict[i] = 1
            else:
                # увеличить значение для этой кухни (ключа) в словаре CuisinesDict
                CuisinesDict[i] += 1


df['cuisine_style'].apply(CuisinesDictFill)

# 4.3.3 среднее кол-во кухонь в одном ресторане
k.groupby('id_ta')['cuisine_style'].nunique().sort_values(
    ascending=False).replace(0, 1).mean()

# 4.3.2 какая кухня представлена в наибольшем кол-ве ресторанов
k.groupby(['cuisine_style'])['id_ta'].nunique(
).sort_values(ascending=False).head(1)

# 2.REVIEWS


# Разделим столбец с отзывами на текст и дату

df['reviews'] = df.reviews.str.replace(']', '').str.replace('[', '')
# 4.4.1. Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.
df['reviews'][1]
reviews_table = df['reviews'].str.split(', ', expand=True)
reviews_table[3].dropna(inplace=True)
df['rew_date'] = pd.to_datetime(reviews_table[3], errors='coerce')
df['rew_date1'] = pd.to_datetime(reviews_table[2], errors='coerce')
df['diff'] = df['rew_date1']-df['rew_date']
df['rew_date'] = pd.to_datetime(df['rew_date'], format='%Y-%m-%d')
df['rew_date1'] = pd.to_datetime(df['rew_date1'], format='%Y-%m-%d')
df['diff'] = (df.rew_date1 - df.rew_date).dt.days

# создаю столбец с разницей в отзывах, но поддаю сомнению слишком большую разницу в днях,
# и решаю взять наиболее свежие отзывы
# убираю выбросы

daty = pd.DataFrame({'DATE': [pd.Timestamp(x) for x in df['rew_date']]})
# print(daty)
qa = daty['DATE'].quantile(0.1)  # lower 10%
qb = daty['DATE'].quantile(0.9)  # higher 10%
# remove outliers
df['new_date'] = daty[(daty['DATE'] >= qa) & (daty['DATE'] <= qb)]


da = pd.DataFrame({'DATES': [pd.Timestamp(x) for x in df['rew_date1']]})
# print(daty)
qa = da['DATES'].quantile(0.1)  # lower 10%
qb = da['DATES'].quantile(0.9)  # higher 10%

# Получаю новый признака разницы дат отзывов
df['new_date1'] = da[(da['DATES'] >= qa) & (da['DATES'] <= qb)]

df['new_date'] = pd.to_datetime(df['new_date'], format='%Y-%m-%d')
df['new_date1'] = pd.to_datetime(df['new_date1'], format='%Y-%m-%d')
df['new_diff'] = (df.new_date1 - df.new_date).dt.days

# Заменяю "новизну" отзывов на ранги
df['last_review'] = df.new_date1.dt.year
df['last_review'] = df['last_review'].replace(2017.0, 2)
df['last_review'] = df['last_review'].replace(2016.0, 1)
df['last_review'] = df['last_review'].replace(2018.0, 3)

# Создаю новый признак с разницей между актуальным днем и последним отзывом
df['fix'] = '2020-12-03 00:00'
df['fix'] = pd.DataFrame({'DATES': [pd.Timestamp(x) for x in df['fix']]})
df['last_'] = (df.fix - df.new_date1).dt.days

# Признаки на основании текста отзывов

df['good_rev'] = df['reviews'].str.contains(r'good', na=True)  # ,
df['good_rev'] = df['good_rev'].astype(int)

df['best_rev'] = df['reviews'].str.contains(r'best', na=True)  # ,
df['best_rev'] = df['best_rev'].astype(int)


df['great_rev'] = df['reviews'].str.contains(r'great', na=True)  # ,
df['great_rev'] = df['great_rev'].astype(int)

df['bad_rev'] = df['reviews'].str.contains(r'bad', na=True)  # ,
df['bad_rev'] = df['bad_rev'].astype(int)


# Кол-во на отзывов на душу населения
df['reviews_per_people100'] = df.apply(lambda x:
                                       round(x['number_of_reviews']*10000/x['population'], 4), axis=1)

# 4.4.1. Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.
df['reviews'][1]
reviews_table = df['reviews'].str.split(', ', expand=True)
reviews_table[3].dropna(inplace=True)
df['rew_date'] = pd.to_datetime(reviews_table[3], errors='coerce')
df['rew_date1'] = pd.to_datetime(reviews_table[2], errors='coerce')
df['diff'] = df['rew_date1']-df['rew_date']


# Заменю ценовую категорию на числовые значения
price_range_dict = {'$': 10, '$$ - $$$': 100, '$$$$': 1000}
df['price_range'] = df['price_range'].map(price_range_dict)

# Загружаю новый датасет о наличии мишленовской звезды у ресторанов по странам

mi = pd.read_csv('../input/ta-rest/michelin_stars.csv')
mi = mi.drop(['the_geom', 'cartodb_id',
              'cartodb_georef_status', 'guide_year'], axis=1)
df = df.merge(mi, how='left')

# Загружаю новый датасет о 50 лучших ресторанах и доп информации о ресторанах
best = pd.read_csv('../input/world-best-50-restaurants-2019/50 best restaurants database.csv')
best['name'] = best['Restaurant']
best.columns = map(lambda name: name.lower(), best.columns)
best.columns
b = best.drop(['ranking', 'restaurant', 'lat', 'lon',
               'stars', 'website', 'description'], axis=1)
df = df.merge(b, how='left')

# Загружаю новый датасет, чтобы взять название и id ресторана
ta = pd.read_csv('../input/ta-rest/TA_restaurants_curated.csv')
ta.columns = map(lambda name: name.lower(), ta.columns)
ta.columns
ta = ta.drop(['unnamed: 0', 'city', 'cuisine style', 'ranking', 'rating',
              'price range', 'number of reviews', 'name','reviews', 'url_ta'], axis=1)
df = df.merge(ta, how='left')

# Создаю новый признак о средней цене дегустационного меню в ресторанах,
# заранее переведя все в USD, курс указан на начало декабря
df.currency.value_counts()
currency_dict = {'EUR': 1.21, 'GBP': 1.3481, 'DKK': 0.1629, 'SEK': 0.12}

df['currency_for_usd'] = df['currency'].map(currency_dict)
df['price_menu_in_usd'] = df['currency_for_usd']*df['menu']
df.currency_for_usd.value_counts()

# проведем корреляционный анализ числовых признаков
df_num = df.copy()
df_num = df_num[['ranking', 'rating',
                 'price_range', 'number_of_reviews',
                 'relative_ranking', 'restaurants_count', 'population',
                 'per_capital', 'restaurant_chain', 'num_of_cuis', 'cuisine_count_mean',
                 'veg','new_diff', 'last_review', 'fix', 'last_',
                 'restaurants_with_three_stars', 'restaurants_with_two_stars',
                 'restaurants_with_one_star',  'price_menu_in_usd', 'good_rev', 'best_rev',
                 'great_rev', 'bad_rev', 'reviews_per_people100']]
#df_num.corr()
# и далее рассмотрим по отдельности признаки, наиболее сильно влияющие на оценку по тесту

# наиболее сильная корреляция наблюдается в restaurant_chain, num_of_cuis, ranking 

#Ranking сильно коррелирует с Rating, возможно потому что признание ресторана в городе
#безусловно будет влиять на рейтинг

#Также можно предположить что количетсво типов кухонь в ресторане хорошо влияет на рейтинг, т.к.
#иногда удобно наряду с сытными грузинскими блюдами заказать Healthy food из Veg меню

#Наличие ресторана в сети ресторанов, мне кажется, может сказываться на рейтинге положительно,
#в случае, если четко соблюдается регламент сети. Те же рестораны Fast food, в незнакомом городе
#увидев вывеску KFC, например, сразу вспоминаешь аппетитные куриные ножки и "мысленный" рейтинг ресторана
#увеличивается

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)

In [ ]:
def find_item(cell):
    if item in cell:
        return 1
    return 0


for item in restaurant_chain:
    df_train['restaurant_chain'] = df_train['restaurant_id'].apply(find_item)

# Сколько городов представлено в наборе данных?
len(df['city'].unique())

# CUISINES

# столбец с кухнями разделяем на отдельные виды
df_train['cuisine_style'] = df_train.cuisine_style.str.replace(
    ']', '').str.replace('[', '')
o = df_train[['id_ta', 'cuisine_style']]
o.cuisine_style.fillna("'Europian'", inplace=True)
s = o['cuisine_style'].str.split(',').apply(Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'cuisine_style'
del o['cuisine_style']
k = o.join(s)

# Создаем признак "Кол-во кухонь в каждом ресторане"
table_with_num = k.copy()
table_with_num['num_of_cuis'] = (table_with_num.groupby(
    ['id_ta'])['cuisine_style']).transform('nunique')
table_with_num.drop(['cuisine_style'], axis=1, inplace=True)
table_with_num = table_with_num.drop_duplicates()
df_train = df_train.merge(table_with_num, how='left')

# среднее количество кухонь на ресторан в городе
df_train['cuisine_count_mean'] = df_train['city'].map(
    df_train.groupby('city')['num_of_cuis'].mean())

# Добавляю признак о наличии веганской кухни в ресторане

df_train['veg'] = df_train['cuisine_style'].str.contains(r'Veg', na=True)
df_train['veg'] = df_train['veg'].astype(int)
CuisinesDict = {}
# Функция заполнения словаря CuisinesDict


def CuisinesDictFill(cuisines):
    global CuisinesDict
    # из входящей строки вида "['German', 'Central European', 'Vegetarian Friendly']"
    cuisines = str(cuisines).replace("['", "").replace(
        "']", "").replace("', '", ",").split(',')
    # получили список отдельных значений вида ['German', 'Central European', 'Vegetarian Friendly']
    if cuisines[0] != 'nan':  # если список кухонь не пустой
        for i in cuisines:  # перебор полученного списка
            if CuisinesDict.get(i) == None:  # нет такой кухни в словаре CuisinesDict
                # добавить кухню (ключ) в CuisinesDict со значением 1
                CuisinesDict[i] = 1
            else:
                # увеличить значение для этой кухни (ключа) в словаре CuisinesDict
                CuisinesDict[i] += 1


df_train['cuisine_style'].apply(CuisinesDictFill)

# 4.3.3 среднее кол-во кухонь в одном ресторане
k.groupby('id_ta')['cuisine_style'].nunique().sort_values(
    ascending=False).replace(0, 1).mean()

# 4.3.2 какая кухня представлена в наибольшем кол-ве ресторанов
k.groupby(['cuisine_style'])['id_ta'].nunique(
).sort_values(ascending=False).head(1)

# 2.REVIEWS


# Разделим столбец с отзывами на текст и дату

df_train['reviews'] = df_train.reviews.str.replace(']', '').str.replace('[', '')
# 4.4.1. Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.
df_train['reviews'][1]
reviews_table = df_train['reviews'].str.split(', ', expand=True)
reviews_table[3].dropna(inplace=True)
df_train['rew_date'] = pd.to_datetime(reviews_table[3], errors='coerce')
df_train['rew_date1'] = pd.to_datetime(reviews_table[2], errors='coerce')
df_train['diff'] = df_train['rew_date1']-df_train['rew_date']
df_train['rew_date'] = pd.to_datetime(df_train['rew_date'], format='%Y-%m-%d')
df_train['rew_date1'] = pd.to_datetime(df_train['rew_date1'], format='%Y-%m-%d')
df_train['diff'] = (df_train.rew_date1 - df_train.rew_date).dt.days

# создаю столбец с разницей в отзывах, но поддаю сомнению слишком большую разницу в днях,
# и решаю взять наиболее свежие отзывы
# убираю выбросы

daty = pd.DataFrame({'DATE': [pd.Timestamp(x) for x in df['rew_date']]})
# print(daty)
qa = daty['DATE'].quantile(0.1)  # lower 10%
qb = daty['DATE'].quantile(0.9)  # higher 10%
# remove outliers
df_train['new_date'] = daty[(daty['DATE'] >= qa) & (daty['DATE'] <= qb)]


da = pd.DataFrame({'DATES': [pd.Timestamp(x) for x in df_train['rew_date1']]})
# print(daty)
qa = da['DATES'].quantile(0.1)  # lower 10%
qb = da['DATES'].quantile(0.9)  # higher 10%

# Получаю новый признака разницы дат отзывов
df_train['new_date1'] = da[(da['DATES'] >= qa) & (da['DATES'] <= qb)]

df_train['new_date'] = pd.to_datetime(df_train['new_date'], format='%Y-%m-%d')
df_train['new_date1'] = pd.to_datetime(df_train['new_date1'], format='%Y-%m-%d')
df_train['new_diff'] = (df_train.new_date1 - df_train.new_date).dt.days

# Заменяю "новизну" отзывов на ранги
df_train['last_review'] = df_train.new_date1.dt.year
df_train['last_review'] = df_train['last_review'].replace(2017.0, 2)
df_train['last_review'] = df_train['last_review'].replace(2016.0, 1)
df_train['last_review'] = df_train['last_review'].replace(2018.0, 3)

# Создаю новый признак с разницей между актуальным днем и последним отзывом
df_train['fix'] = '2020-12-03 00:00'
df_train['fix'] = pd.DataFrame({'DATES': [pd.Timestamp(x) for x in df_train['fix']]})
df_train['last_'] = (df_train.fix - df_train.new_date1).dt.days

# Признаки на основании текста отзывов

df_train['good_rev'] = df_train['reviews'].str.contains(r'good', na=True)  # ,
df_train['good_rev'] = df_train['good_rev'].astype(int)

df_train['best_rev'] = df_train['reviews'].str.contains(r'best', na=True)  # ,
df_train['best_rev'] = df_train['best_rev'].astype(int)


df_train['great_rev'] = df_train['reviews'].str.contains(r'great', na=True)  # ,
df_train['great_rev'] = df_train['great_rev'].astype(int)

df_train['bad_rev'] = df_train['reviews'].str.contains(r'bad', na=True)  # ,
df_train['bad_rev'] = df_train['bad_rev'].astype(int)


# Кол-во на отзывов на душу населения
df_train['reviews_per_people100'] = df_train.apply(lambda x:
                                       round(x['number_of_reviews']*10000/x['population'], 4), axis=1)

# 4.4.1. Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.
df_train['reviews'][1]
reviews_table = df_train['reviews'].str.split(', ', expand=True)
reviews_table[3].dropna(inplace=True)
df_train['rew_date'] = pd.to_datetime(reviews_table[3], errors='coerce')
df_train['rew_date1'] = pd.to_datetime(reviews_table[2], errors='coerce')
df_train['diff'] = df_train['rew_date1']-df_train['rew_date']


# Заменю ценовую категорию на числовые значения
price_range_dict = {'$': 10, '$$ - $$$': 100, '$$$$': 1000}
df_train['price_range'] = df_train['price_range'].map(price_range_dict)

# Загружаю новый датасет о наличии мишленовской звезды у ресторанов по странам

mi = pd.read_csv('../input/ta-rest/michelin_stars.csv')
mi = mi.drop(['the_geom', 'cartodb_id',
              'cartodb_georef_status', 'guide_year'], axis=1)
df_train = df_train.merge(mi, how='left')

# Загружаю новый датасет о 50 лучших ресторанах и доп информации о ресторанах
best = pd.read_csv('../input/world-best-50-restaurants-2019/50 best restaurants database.csv')
best['name'] = best['Restaurant']
best.columns = map(lambda name: name.lower(), best.columns)
best.columns
b = best.drop(['ranking', 'restaurant', 'lat', 'lon',
               'stars', 'website', 'description'], axis=1)
df_train = df_train.merge(b, how='left')

# Загружаю новый датасет, чтобы взять название и id ресторана
ta = pd.read_csv('../input/ta-rest/TA_restaurants_curated.csv')
ta.columns = map(lambda name: name.lower(), ta.columns)
ta.columns
ta = ta.drop(['unnamed: 0', 'city', 'cuisine style', 'ranking', 'rating',
              'price range', 'number of reviews', 'name','reviews', 'url_ta'], axis=1)
df_train = df_train.merge(ta, how='left')

# Создаю новый признак о средней цене дегустационного меню в ресторанах,
# заранее переведя все в USD, курс указан на начало декабря
df_train.currency.value_counts()
currency_dict = {'EUR': 1.21, 'GBP': 1.3481, 'DKK': 0.1629, 'SEK': 0.12}

df_train['currency_for_usd'] = df_train['currency'].map(currency_dict)
df_train['price_menu_in_usd'] = df_train['currency_for_usd']*df_train['menu']



In [ ]:
data = df.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
df.info()

In [ ]:
df=df.truncate(before=1, after=10000)

In [ ]:
df.isnull().sum()



In [ ]:
#df = pd.get_dummies(df, columns=['city'], dummy_na=True)

In [ ]:
#df_train=pd.get_dummies(df_train, columns=['city'], dummy_na=True)

In [ ]:
df['price_range'].fillna(0, inplace=True)

In [ ]:
df_train['price_range'].fillna(0, inplace=True)

In [ ]:

df_train['number_of_reviews'].fillna(df_train['number_of_reviews'].mean(), inplace=True)

In [ ]:

df['number_of_reviews'].fillna(df['number_of_reviews'].mean(), inplace=True)

In [ ]:
df.new_diff.fillna(df['new_diff'].mean(), inplace=True)

In [ ]:
df_train.new_diff.fillna(df_train['new_diff'].mean(), inplace=True)

In [ ]:
df.last_review .fillna(df['last_review'].mean(), inplace=True)

In [ ]:
df_train.last_review .fillna(df_train['last_review'].mean(), inplace=True)

In [ ]:
df.last_.fillna(df['last_'].mean(), inplace=True)

In [ ]:
df_train.last_.fillna(df_train['last_'].mean(), inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df_train.isnull().sum()

In [ ]:
df = df.drop(['restaurant_id','city', 'cuisine_style',
             'reviews', 'rew_date', 'rew_date1', 'fix', 'diff', 'bad_rev','new_date', 'new_date1', 'url_ta', 'id_ta', 'chef', 'name', 'menu', 'currency'
             , 'currency_for_usd','total','country'], axis=1)


In [ ]:
df_train = df_train.drop(['restaurant_id', 'city','cuisine_style',
             'reviews', 'rew_date', 'rew_date1', 'fix', 'diff', 'bad_rev','new_date', 'new_date1', 'url_ta', 'id_ta', 'chef', 'name', 'menu', 'currency'
             , 'currency_for_usd','total','country'], axis=1)



In [ ]:
df_train.fillna(0, inplace=True)

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df_train.isnull().sum()

In [ ]:
data = df.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:

#y = train_data.Rating.values
#X['number_of_reviews'].fillna(X['number_of_reviews'].mean(), inplace=True)
#X['new_diff'].fillna(X['new_diff'].min(), inplace=True)
#
#X['last_review'].fillna(1, inplace=True)
#X.last_.fillna(X['last_'].mean(), inplace=True)
#X.reviews_per_people100.fillna(X['reviews_per_people100'].mean(), inplace=True)
#X['restaurants_with_three_stars'].fillna(0, inplace=True)
#X['restaurants_with_two_stars'].fillna(0, inplace=True)
#X['restaurants_with_one_star'].fillna(0, inplace=True)
#X['price_menu_in_usd'].fillna(X['price_menu_in_usd'].mean(), inplace=True)
#X.dropna(axis='index', how='any', subset=['price_range'])
#X.columns

In [ ]:
df.info()

In [ ]:
df.fillna(0,inplace=True)

In [ ]:
df_train.fillna(0,inplace=True)

In [ ]:
data.fillna(0,inplace=True)

In [ ]:
data.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)

test_data = data.query('sample == 0').drop(['sample'], axis=1)


In [ ]:
test_data

In [ ]:
y= train_data.rating.values            # наш таргет
X = train_data.drop(['rating'], axis=1)


**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['rating'], axis=1)

In [ ]:
test_data

In [ ]:
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')
sample_submission

In [ ]:
sample_submission.columns= map(lambda name: name.lower(), sample_submission.columns)

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
sample_submission['rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
